In [ ]:
# This is to check the catalog of Whitten+, as sent by T. Beers on Nov. 26

# Created 2023 Jan. 3 by E.S.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib qt

In [2]:
stem = "/Users/bandari/Documents/git.repos/rrlfe/"

In [3]:
# read in Whitten+ retrievals

df_whitten_stars = pd.read_csv("./data/SPLUS_STRIPE82_ebv_retrain.csv")

In [4]:
# functionality to read in our SDSS star IDs, remove repeats, and write back out 
# so that they can be fed into SDSS SkyServer 

'''
# 
df_mine = pd.read_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", delim_whitespace=True)
df_mine_no_repeats = df_mine.drop_duplicates()
df_mine_no_repeats.to_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", sep=" ", index=False)
df_mine_no_repeats = pd.read_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", delim_whitespace=True)
'''

'\n# \ndf_mine = pd.read_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", delim_whitespace=True)\ndf_mine_no_repeats = df_mine.drop_duplicates()\ndf_mine_no_repeats.to_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", sep=" ", index=False)\ndf_mine_no_repeats = pd.read_csv(stem + "notebooks_for_development/data/spalding_sdss_spectra_for_xmatch_norepeats.csv", delim_whitespace=True)\n'

In [5]:
# read in RA, DEC info for our stars, as returned by SDSS SkyServer

df_our_sdss_stars = pd.read_csv(stem + "notebooks_for_development/data/Skyserver_CrossID1_4_2023_5_21_47_AM.csv")

In [6]:
# give some cols same name, so we can merge on them

df_whitten_stars["ra"] = df_whitten_stars["RA"]
df_whitten_stars["dec"] = df_whitten_stars["Dec"]

# some rounding is necessary to enable matching
df_whitten_stars["ra_round"] = np.round(df_whitten_stars["ra"],2)
df_our_sdss_stars["ra_round"] = np.round(df_our_sdss_stars["ra"],2)
df_whitten_stars["dec_round"] = np.round(df_whitten_stars["dec"],3)
df_our_sdss_stars["dec_round"] = np.round(df_our_sdss_stars["dec"],3)

In [7]:
# this merges tables based on RA, DEC, and leaves the Whitten FeH in the final table
# (but our FeH is still missing)

merged_df_whitten_feh = pd.merge(df_whitten_stars, df_our_sdss_stars, how="inner", on=["ra_round","dec_round"])

In [8]:
# read in our retrieved Fe/H and splice stuff to enable matching

df_our_data = pd.read_csv(stem + "rrlfe_io_20221220_sdss_test/bin/retrieved_vals.csv")

In [9]:
split_1 = df_our_data["orig_spec_file_name"].str.split("-", 3, expand=True)
split_2 = split_1[3].str.split("_", 1, expand=True)

In [10]:
df_our_data["plate"] = split_1[1].astype(int)
df_our_data["mjd"] = split_1[2].astype(int)
df_our_data["fiberid"] = split_2[0].astype(int)

In [42]:
# our stars can have multiple FeHs, because of multiple epochs
# (an 'inner' merge in this context means that repeats are allowed, but any rows with NaNs 
# left in either FeH column is removed)

merged_df_whitten_and_our_feh = pd.merge(df_our_data, merged_df_whitten_feh, on=["plate", "mjd", "fiberid"], 
                                         how="inner")

In [45]:
plt.scatter(merged_df_whitten_and_our_feh["feh_retrieved"],merged_df_whitten_and_our_feh["NET_FEH"])
plt.plot([-4.0,4.0],[-4.0,4.0], linestyle="--")
plt.xlabel("Our FeH")
plt.ylabel("Whitten+ FeH")
plt.show()